# $sortByCount 的基本操作

In [ ]:
# find one company document
db.companies.findOne()

# create text index
db.companies.createIndex({ description: "text", overview: "text" })

In [ ]:
# find companies matching term `networking` using text search
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "network" }
        }
    }
]).pretty()

In [ ]:
# $sortByCount single query facet for the previous search
# 當然也可以使用操作符 $group。
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "network" }
        }
    },
    { $sortByCount: "$category_code" }
]).pretty()

In [ ]:
# extend the pipeline for a more elaborate facet
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "network" }
        }
    },
    {
        $unwind: "$offices"
    },
    {
        $match: { "offices.city": { $ne: "" }}
    },
    {
        $sortByCount: "$offices.city"
    }
]).pretty()

### 範例01

In [ ]:
# performing a group followed by a sort to rank occurence
db.movies.aggregate([
    {
        $group: {
            _id: "$imdb.rating",
            count: { $sum: 1}
        }
    },
    { $sort: { count: -1 }}
])

In [ ]:
# sortByCount is equivalent to the above. In fact, if you execute this pipeline
# with { explain: true } you will see that it is transformed to the above!
db.movies.aggregate([
    {
        $sortByCount: "$imdb.rating"
    }
])